# importing, manipulating, and representing data 

The basis of any statistical analysis is the underlying data.

A data-set is typically presented as a file containing information formatted as a table:
 * each line correspond to an observation ( individual, sample, ... )
 * each column correspond to a measured variable ( height, sex, gene expression, ... )


To read data file and manipulate the data, we will rely on [pandas](https://pandas.pydata.org/)
Pandas is a "high-level" module, designed for statistics/exploratory analysis.
A great strength of pandas is its **DataFrame** which emulates many of the convenient behavior and syntax of their eponym counterpart in the **R** language.


To graphically represent the data, we will rely on [seaborn](https://seaborn.pydata.org/index.html).
Seaborn is designed to work hand-in-hand with pandas DataDrame to produce **efficient data representation** from fairly simple commands. They propose very good tutorials as well as a gallery (with associated code) that can get you started quickly.


In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
print('OK')

OK


In [ ]:
# ToC

1. Reading the data <a id='reading'></a>

    1.1. the basics <a id='reading.1'></a>

    1.2. header or not header, that is the question <a id='reading.2'></a>

    1.3. setting up the row index <a id='reading.3'></a>

    1.4. other options <a id='reading.4'></a>

    1.5. more formats <a id='reading.5'></a>
    


## 1. Reading the data <a id='reading'></a>

### 1.1. the basics <a id='reading.1'></a>

what is the file name? location?
what is the saprator between fields??


`pd.read_table` is a generalistic function to read table. Aside from the name of the file to read, here are some useful parameters:
* `sep` : separator between columns (by default '\t')
* `header` : Row number(s) to use as the column names. By default it will use the first line as a header. use `header=None` if the file does not contain column name.
* `skiprows` : Line numbers to skip (0-indexed) or number of lines to skip (int) at the start of the file.
* (true_values/false_values ??)

Of course you can learn (much) more using `help(pd.read_table)`.



Let's try to load the `data/titanic.csv` file. As its name suggest, this table contains data about the ill-fated [Titanic](https://en.wikipedia.org/wiki/Titanic) passengers, travelling from England to New York in April 1912.

The data-file is named `"titanic.csv"` and like its extension suggests, it contains **C**omma-**S**eparated **V**alues.


In [15]:
import pandas as pd

df = pd.read_table( "data/titanic.csv" ) 
#try to see what happens when sep has a different value

df.head() # this returns the 6 first lines of the table

,"Name,Sex,Age,Pclass,Survived,Family,Fare,Embarked"
0,"Bjornstrom-Steffansson Mr. Mauritz Hakan,male,..."
1,"Coleff Mr. Peju,male,36,3,0,0,7.5,S"
2,"Laroche Miss. Simonne Marie Anne Andree,female..."
3,"Smith Miss. Marion Elsie,female,40,2,1,0,13,S"
4,"Dooley Mr. Patrick,male,32,3,0,0,7.75,Q"


It does not look so great...

**micro-exercise** : try to fix the cell just above by playing with the option(s) of `pd.read_table`


In [3]:
help(pd.read_table)

Help on function read_table in module pandas.io.parsers:

read_table(filepath_or_buffer: Union[ForwardRef('PathLike[str]'), str, IO[~T], io.RawIOBase, io.BufferedIOBase, io.TextIOBase, _io.TextIOWrapper, mmap.mmap], sep=<object object at 0x7feb206fe210>, delimiter=None, header='infer', names=None, index_col=None, usecols=None, squeeze=False, prefix=None, mangle_dupe_cols=True, dtype=None, engine=None, converters=None, true_values=None, false_values=None, skipinitialspace=False, skiprows=None, skipfooter=0, nrows=None, na_values=None, keep_default_na=True, na_filter=True, verbose=False, skip_blank_lines=True, parse_dates=False, infer_datetime_format=False, keep_date_col=False, date_parser=None, dayfirst=False, cache_dates=True, iterator=False, chunksize=None, compression='infer', thousands=None, decimal: str = '.', lineterminator=None, quotechar='"', quoting=0, doublequote=True, escapechar=None, comment=None, encoding=None, dialect=None, error_bad_lines=True, warn_bad_lines=True, delim_


<br>

<br>

So you have seen that by default, `read_table()` expects the input data to be **tab-delimited**, but since this is not the case of the `titanic.csv` file, each line was treated as a single field (column), thus creating a DataFrame with a single column.

As implied by its `.csv` extension (for "comma-separeted values"), the `titanic.csv` file contains **comma-delimited** values. To load a CSV file, we can either:
* Specify the separator value in `read_table(sep=",")`.
* Use `read_csv()`, a function that will use comma as separator by default.

In [16]:
import pandas as pd

df = pd.read_table( "data/titanic.csv"  ,sep = ',' ) 
# alternatively : df = pd.read_csv( "data/titanic.csv"  ,sep = ',' ) 

df.head() # this returns the 6 first lines of the table

,Name,Sex,Age,Pclass,Survived,Family,Fare,Embarked
0,Bjornstrom-Steffansson Mr. Mauritz Hakan,male,28.0,1,1,0,26.55,S
1,Coleff Mr. Peju,male,36.0,3,0,0,7.50,S
2,Laroche Miss. Simonne Marie Anne Andree,female,3.0,2,1,1,41.58,C
3,Smith Miss. Marion Elsie,female,40.0,2,1,0,13.00,S
4,Dooley Mr. Patrick,male,32.0,3,0,0,7.75,Q


### 1.2. header or not header, that is the question <a id='reading.2'></a>


Another important aspect of reading data is whether your dataset has a header or not. 
By default, `pd.read_table` will expect the first line to  be a header, unless you either :
 * use the argument `header=None` 
 * specify column names using the `names` argument


In [17]:
df = pd.read_table( "data/titanic_no_header.csv"  ,sep = ',' ) 
df.head(n=3) 

,Bjornstrom-Steffansson Mr. Mauritz Hakan,male,28,1,1.1,0,26.55,S
0,Coleff Mr. Peju,male,36.0,3,0,0,7.50,S
1,Laroche Miss. Simonne Marie Anne Andree,female,3.0,2,1,1,41.58,C
2,Smith Miss. Marion Elsie,female,40.0,2,1,0,13.00,S


Notice how the **first entry was set as column names**... that is not ideal.

Let's correct this:

In [18]:
df = pd.read_table( "data/titanic_no_header.csv"  ,sep = ',' , header = None) 
df.head(n=3) 

,0,1,2,3,4,5,6,7
0,Bjornstrom-Steffansson Mr. Mauritz Hakan,male,28.0,1,1,0,26.55,S
1,Coleff Mr. Peju,male,36.0,3,0,0,7.50,S
2,Laroche Miss. Simonne Marie Anne Andree,female,3.0,2,1,1,41.58,C


Much better! 

Let's go one step further and assign our own column names :

In [19]:
df = pd.read_table( "data/titanic_no_header.csv"  ,sep = ',' , 
                   names = ['name','column2','age','column4','blip','bloop','spam','eggs']) 
# as you can see, we can choose our own name, whether they make sense or not
df.head(n=3) 

,name,column2,age,column4,blip,bloop,spam,eggs
0,Bjornstrom-Steffansson Mr. Mauritz Hakan,male,28.0,1,1,0,26.55,S
1,Coleff Mr. Peju,male,36.0,3,0,0,7.50,S
2,Laroche Miss. Simonne Marie Anne Andree,female,3.0,2,1,1,41.58,C


### 1.3. setting up the row index <a id='reading.3'></a>

Now that we have set up column names, let's see how to setup row names, called the **index**.

> not all dataset need an index. oftentimes the default numbered lines is enough.

Again, we have several options at our disposal.

**1. the data file has one less column names that column data**


In [28]:
!head -n 4 data/titanic_implicit_index.csv

Sex,Age,Pclass,Survived,Family,Fare,Embarked
Bjornstrom-Steffansson Mr. Mauritz Hakan,male,28,1,1,0,26.55,S
Coleff Mr. Peju,male,36,3,0,0,7.5,S
Laroche Miss. Simonne Marie Anne Andree,female,3,2,1,1,41.58,C


In [26]:
df = pd.read_table( "data/titanic_implicit_index.csv"  ,sep = ',' ) 
df.head(n=3) 

,Sex,Age,Pclass,Survived,Family,Fare,Embarked
Bjornstrom-Steffansson Mr. Mauritz Hakan,male,28.0,1,1,0,26.55,S
Coleff Mr. Peju,male,36.0,3,0,0,7.50,S
Laroche Miss. Simonne Marie Anne Andree,female,3.0,2,1,1,41.58,C


In [22]:
df.index[:5] ## access the index directly

Index(['Bjornstrom-Steffansson Mr. Mauritz Hakan', 'Coleff Mr. Peju',
       'Laroche Miss. Simonne Marie Anne Andree', 'Smith Miss. Marion Elsie',
       'Dooley Mr. Patrick'],
      dtype='object')

**In that case the first, nameless, column is used as index**

**2. we specify the index using `index_col`**

In [31]:
df = pd.read_table( "data/titanic.csv"  ,sep = ',' , index_col = 0 )
# alternatively , specify the index column by name : index_col = 'Name'
df.head(n=3) 

,Sex,Age,Pclass,Survived,Family,Fare,Embarked
Name,,,,,,,
Bjornstrom-Steffansson Mr. Mauritz Hakan,male,28.0,1,1,0,26.55,S
Coleff Mr. Peju,male,36.0,3,0,0,7.50,S
Laroche Miss. Simonne Marie Anne Andree,female,3.0,2,1,1,41.58,C


> Note that pandas also has a system of multiple, hierarchised indexing. This is, however, a much more specialized and advanced feature.

### 1.4. other options <a id='reading.4'></a>

`pd.read_table` has a vast arrays of option.
We cannot go though all of the them, but here are a few which may be of interest to you:

* `true_values`/`false_values`, each a list. a must if you have columns encoded with "yes"/"no" labels.
* `na_values` : takes a list. Ideal when your NAs are encoded as something unusual (eg, `.`,` `,`-9999`,...)
* `parse_dates`/`infer_datetime_format`/`date_parser` : options to help you handle date parsing, which can oitherwise be a nightmare. [more on this](https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html)
* `compression` : your data is in a compressed (zip, gzip, ...), not a problem!



### 1.5. more formats <a id='reading.5'></a>

As you might expect, pandas is not limited to text, csv/tsv-like files.

* `pd.read_excel()`
* `pd.read_json()`
* `pd.read_sql()` 
* ... see [here for an exhaustive list of pandas reader and writer functions](https://pandas.pydata.org/pandas-docs/stable/user_guide/io.html).



In [39]:
genbank_df = pd.read_json( 'data/genbank.sub.ndjson' , lines = True )
genbank_df.head()

,genbank_accession,genbank_accession_rev,database,strain,region,location,collected,submitted,length,host,isolation_source,biosample_accession,title,authors,publications,sequence
0,MW553299,MW553299.1,GenBank,SARS-CoV-2/human/ARG/Cordoba-189-251/2020,South America,Argentina,2020-05-16,2021-02-01T00:00:00Z,29719,Homo sapiens,oronasopharynx,,Severe acute respiratory syndrome coronavirus ...,Direct Submission,,GATCTGTTCTCTAAACGAACTTTAAAATCTGTGTGGCTGTCACTCG...
1,MW553294,MW553294.1,GenBank,SARS-CoV-2/human/ARG/Cordoba-2635-202/2020,South America,Argentina,2020-06-04,2021-02-01T00:00:00Z,29723,Homo sapiens,oronasopharynx,,Severe acute respiratory syndrome coronavirus ...,Direct Submission,,GATCTGTTCTCTAAACGAACTTTAAAATCTGTGTGGCTGTCACTCG...
2,MW553295,MW553295.1,GenBank,SARS-CoV-2/human/ARG/Cordoba-2842-202/2020,South America,Argentina,2020-06-04,2021-02-01T00:00:00Z,29723,Homo sapiens,oronasopharynx,,Severe acute respiratory syndrome coronavirus ...,Direct Submission,,GATCTGTTCTCTAAACGAACTTTAAAATCTGTGTGGCTGTCACTCG...
3,MW553296,MW553296.1,GenBank,SARS-CoV-2/human/ARG/Cordoba-1083-6/2020,South America,Argentina,2020-06-04,2021-02-01T00:00:00Z,29717,Homo sapiens,oronasopharynx,,Severe acute respiratory syndrome coronavirus ...,Direct Submission,,TCTCTAAACGAACTTTAAAATCTGTGTGGCTGTCACTCGGCTGCAT...
4,MW553297,MW553297.1,GenBank,SARS-CoV-2/human/ARG/Cordoba-11419-61/2020,South America,Argentina,2020-06-04,2021-02-01T00:00:00Z,29724,Homo sapiens,oronasopharynx,,Severe acute respiratory syndrome coronavirus ...,Direct Submission,,GATCTGTTCTCTAAACGAACTTTAAAATCTGTGTGGCTGTCACTCG...



<br>

**micro-exercise :** read the file `data/pbmc_data.countMatrix.50.txt.zip` as a DataFrame. Determine which is the separator, and decide whether there is a header and/or an index column.

In [55]:
df_sc = pd.read_table('data/pbmc_data.countMatrix.50.txt.zip',sep=' ', index_col = 0)
df_sc.head()

,AAACATACAACCAC,AAACATTGAGCTAC,AAACATTGATCAGC,AAACCGTGCTTCCG,AAACCGTGTATGCG,AAACGCACTGGTAC,AAACGCTGACCAGT,AAACGCTGGTTCTT,AAACGCTGTAGCCA,AAACGCTGTTTCTG,...,AAATTCGAGGAGTG,AAATTCGATTCTCA,AAATTGACACGACT,AAATTGACTCGCTC,AACAAACTCATTTC,AACAAACTTTCGTT,AACAATACGACGAG,AACACGTGCAGAGG,AACACGTGGAAAGT,AACACGTGGAACCT
gene,,,,,,,,,,,,,,,,,,,,,
MIR1302-10,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
FAM138A,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
OR4F5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
RP11-34P13.7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
RP11-34P13.8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
